<img align="left" src="imgs/fonduer-logo.png" width="100px" style="margin-right:20px">

# Tutorial: Generating Candidates from Richly Formatted Data

## Running locally?

If you're running this tutorial interactively on your own machine, you'll need to create a new PostgreSQL database named `intro_candidates`.

If you already have the database `intro_candidates` in your postgresql, please uncomment the first line to drop it. Otherwise, download our database snapshots by executing `./download_data.sh` in the intro tutorial directory.

In [1]:
#! dropdb --if-exists intro_candidates
! createdb intro_candidates
! psql intro_candidates < data/intro_candidates.sql > /dev/null

# Generating Candidates from Richly Formatted Data

A `Candidate` object represents a potential instance of a fact that you would like to extract from your data. For example, if we were trying to extract a `(Part Number, Storage Temperature)` tuple from transistor datasheets, a `Candidate` may be a mention of a `Part Number` found in a header, and a mention of numerical value found in a Table cell. In this tutorial, we will show you first how you _define_ a relation such as the part number and storage temperature example above. And how you then provide _matchers_ and _throttlers_ to generate candidates from the Fonduer Data Model. 

To kick things off, we first connect to the `intro_candidates` database that we imported using the `Meta` class of Fonduer.

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging
from pprint import pprint

from fonduer import Meta, init_logging

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "intro_candidates"
conn_string = f'postgresql://localhost:5432/{ATTRIBUTE}'

# Configure logging for Fonduer
init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

[2019-04-01 21:58:33,697][INFO] fonduer.meta:50 - Setting logging directory to: logs/2019-04-01_21-58-33
[2019-04-01 21:58:33,741][INFO] fonduer.meta:134 - Connecting user:None to localhost:5432/intro_candidates
[2019-04-01 21:58:34,877][INFO] fonduer.meta:161 - Initializing the storage schema


## Defining Relations

All Fonduer applications require the user to provide a custom `Candidate` class definition.

In this tutorial, we define a `Part_Attr` relation to represent a relationship between a a transistor part number and some electrical attribute of that particular transistor.

<img src="imgs/part_attr.png" width="220px">

This `Candidate` is made of two `Mention`s, a `Part` and an `Attr`. Each `Mention` is made up of a`Span` of text (i.e., sequences of words or characters) that represent the mention of a part number and the mention of a maximum storage temperature. We start by defining the `Mentions` which will make up the `Candidate`

In [3]:
from fonduer.candidates.models import mention_subclass

Part = mention_subclass("Part")
Attr = mention_subclass("Attr")

In many traditional knowledge base construction systems, these two spans may come from the same `Sentence`. For example, if we were looking for a `Spouse` relation, we might have a candidate like this one from [Snorkel](https://github.com/HazyResearch/snorkel/):

<img src="https://github.com/HazyResearch/snorkel/raw/f1712dc794711c6174018f9658227b9353904fac/tutorials/workshop/imgs/sentence.jpg" width="700px">

However, in Fonduer, we target _richly formatted data_. When working with richly formatted data, relations rarely come from the same Sentence. Instead, they can come from distant parts of a document like a `Span` of text in an page header paired with a numerical value found in a table dozens of pages later.

<img src="imgs/context.png" width="650px">

## Generating Mentions

Now that we've defined the `Mention` classes we want to extract, the next step is generating those candidates. To do so, Fonduer allows users to provide _matchers_. These inputs are valuable because with richly formatted data, a naive cross-product of all `Mentions` of text in a document would result in an intractable, combinatorial explosion of candidates. Matchers serve to limit the number of `Mentions` generated. Matchers operate on individual `Spans`.

### Matchers
One convenient way to think about matchers is to think of them as a way to define what each component of your relation is. In our example, we can provide a matcher to define what a `part` looks like, and a matcher to define what a valid `attr` looks like (in our case this means definine what a valid maximum storage temperature looks like). 

Fonduer provides some pre-built matchers you can use to help make this easier as documented on [Read the Docs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#matchers). For example, Fonduer provides ways to leverage dictionaries, RegEx, NLP Tags, or arbitrary functions. 

Importantly, matchers should try to be **as specific as possible** while still maintaining high recall.

#### Writing a simple temperature matcher
For the `attr` mention, we are looking for maximum storage temperatures. By inspecting our data (or relying on some domain experience), we have come to the conclusion that maximum storage temperatures are expressed as integers in the range 150 to 205, and only appear as multiples of 5. We can easily express this pattern as a regular expression.

In [4]:
from fonduer.candidates.matchers import RegexMatchSpan

attr_matcher = RegexMatchSpan(rgx=r'(?:[1][5-9]|20)[05]')

#### Writing an advanced transistor part matcher
In contrast, transistor part numbers are complex expressions. For this tutorial, suppose they are complex enough that we actually want to tackle a definition using a few different angles. In this case, we want to leverage:
1. Common [naming conventions](https://en.wikipedia.org/wiki/Transistor#Part_numbering_standards.2Fspecifications) defined by manufacturers as regular expressions
2. A dictionary of part numbers
3. A user-defined function

To start, let's construct a regular expression matcher that captures the naming conventions linked above.

In [5]:
### Transistor Naming Conventions as Regular Expressions ###
eeca_rgx = r'([ABC][A-Z][WXYZ]?[0-9]{3,5}(?:[A-Z]){0,5}[0-9]?[A-Z]?(?:-[A-Z0-9]{1,7})?(?:[-][A-Z0-9]{1,2})?(?:\/DG)?)'
jedec_rgx = r'(2N\d{3,4}[A-Z]{0,5}[0-9]?[A-Z]?)'
jis_rgx = r'(2S[ABCDEFGHJKMQRSTVZ]{1}[\d]{2,4})'
others_rgx = r'((?:NSVBC|SMBT|MJ|MJE|MPS|MRF|RCA|TIP|ZTX|ZT|ZXT|TIS|TIPL|DTC|MMBT|SMMBT|PZT|FZT|STD|BUV|PBSS|KSC|CXT|FCX|CMPT){1}[\d]{2,4}[A-Z]{0,5}(?:-[A-Z0-9]{0,6})?(?:[-][A-Z0-9]{0,1})?)'

part_rgx = '|'.join([eeca_rgx, jedec_rgx, jis_rgx, others_rgx])
part_rgx_matcher = RegexMatchSpan(rgx=part_rgx, longest_match_only=True)

Next, suppose that we have been provided a dictionary of known part numbers (e.g. from crowdsourcing or a collaborator data source). We can use a `DictionaryMatch` which will only match `Spans` of text that appear in the dictionary provided. In our case, we have a transistor part number dictionary from Digikey.com.

In [6]:
import csv
from fonduer.candidates.matchers import DictionaryMatch

def get_digikey_parts_set(path):
    """
    Reads in the digikey part dictionary and yeilds each part.
    """
    all_parts = set()
    with open(path, "r") as csvinput:
        reader = csv.reader(csvinput)
        for line in reader:
            (part, url) = line
            all_parts.add(part)
    return all_parts

### Dictionary of known transistor parts ###
dict_path = 'data/digikey_part_dictionary.csv'
part_dict_matcher = DictionaryMatch(d=get_digikey_parts_set(dict_path))

Futhermore, we can provide user-defined functions as matchers as well! As an example, here we use patterns we notice in document filenames as an indication of whether a `Span` of text is a valid transistor part number. This is particularly useful in a dataset where files are named after their parts (e.g. `bc546.pdf`).

Note that in the code below, we also demonstrate how to use the `Intersect` class to perform an intersection of two different matchers.

In [7]:
from builtins import range
from fonduer.candidates.matchers import LambdaFunctionMatcher, Intersect

def common_prefix_length_diff(str1, str2):
    for i in range(min(len(str1), len(str2))):
        if str1[i] != str2[i]:
            return min(len(str1), len(str2)) - i
    return 0

def part_file_name_conditions(attr):
    file_name = attr.sentence.document.name
    if len(file_name.split('_')) != 2: return False
    if attr.get_span()[0] == '-': return False
    name = attr.get_span().replace('-', '')
    return any(char.isdigit() for char in name) and any(char.isalpha() for char in name) and common_prefix_length_diff(file_name.split('_')[1], name) <= 2

add_rgx = '^[A-Z0-9\-]{5,15}$'

part_file_name_lambda_matcher = LambdaFunctionMatcher(func=part_file_name_conditions)
part_file_name_matcher = Intersect(RegexMatchSpan(rgx=add_rgx, longest_match_only=True), part_file_name_lambda_matcher)

At this point, we've created three separate `Matchers` which we want to combine into a single `Matcher` that can be used to define the `part` component of our `Part_Attr` class. We can do so using the `Union` class.

In [8]:
from fonduer.candidates.matchers import Union

part_matcher = Union(part_rgx_matcher, part_dict_matcher, part_file_name_matcher)

Thus, the `attr_matcher` and `part_matcher` define each component of our relation schema.

## Define Mentions's MentionSpaces
Next, in order to define the "space" of all candidates that are even considered from the document, we need to define a `MentionSpace` for each component of the relation we wish to extract.

In the case of transistor part numbers, the `MentionSpace` can be quite complex due to the need to handle implicit part numbers that are implied in text like "BC546A/B/C...BC548A/B/C", which refers to 9 unique part numbers. To handle these, we consider all n-grams up to 3 words long.

In contrast, the `MentionSpace` for temperature values is simpler: we only need to process different unicode representations of a (-), and don't need to look at more than two words at a time.

When no special preproessing like this is needed, we could have used the default OmniNgrams class provided by fonduer. For example, if we were looking to match polarities, which only take the form of "NPN" or "PNP", we could've used attr_ngrams = MentionNgrams(n_max=1).

In [9]:
from fonduer.candidates import MentionNgrams

part_ngrams = MentionNgrams(n_max=3)
attr_ngrams = MentionNgrams(n_max=2)

Now, we're ready to generate our Mentions using the Matchers and MentionSpaces defined above.

In [10]:
from fonduer.parser.models import Document, Sentence
from fonduer.candidates.models import Mention
from fonduer.candidates import MentionExtractor

docs = session.query(Document).all()

mention_extractor = MentionExtractor(
    session,
    [Part, Attr],
    [part_ngrams, attr_ngrams],
    [part_matcher, attr_matcher],
    parallelism=PARALLEL
)
mention_extractor.apply(docs)
print(f"Num Mentions: {session.query(Mention).count()}")

[2019-04-01 21:58:35,610][INFO] fonduer.candidates.mentions:460 - Clearing table: part
[2019-04-01 21:58:35,614][INFO] fonduer.candidates.mentions:460 - Clearing table: attr
[2019-04-01 21:58:35,617][INFO] fonduer.utils.udf:54 - Running UDF...



Num Mentions: 313


## Candidate Extraction
Next, we can extract our `Candidates`. First, we need to define our `Candidate` as a tuple of `Mentions`.

In [11]:
from fonduer.candidates.models import candidate_subclass

PartAttr = candidate_subclass("PartAttr", [Part, Attr])

### Throttlers
`Throttlers` allow us to further prune excess candidates and avoid annecessarily materializing invalid candidates. But, unlike `Matchers`, which operate on `Mentions`, `Throttlers` operate on `Candidates`. Like `Matchers`, `Throttlers` act as hard filters and should be as specific as possible while maintining complete recall, if possible.

Because `Throttlers` operate on `Candidates`, users can leverage the `data_model_utils` functions provided by Fonduer to write throttling functions using information from multiple modalites of the data. Check the full `data_model_utils` API on [Read the Docs](http://fonduer.readthedocs.io/en/stable/user/data_model_utils.html).

To make this concrete, here we create a `Throttler` that discards candidates if they are in the same `Table`, but the `part` and `attr` are not vertically or horizontally aligned.

In [12]:
import re
from fonduer.utils.data_model_utils import *

def stg_temp_filter(c):
    (part, attr) = c
    if same_table((part, attr)):
        return (is_horz_aligned((part, attr)) or is_vert_aligned((part, attr)))
    return True

temp_throttler = stg_temp_filter

## Running the `CandidateExtractor`

Now, we have all the component necessary to perform candidate extraction. We have defined the "space" of things to consider for each candidate, provided matchers that signal when a valid mention is seen, and a throttler to prunes away excess candidates. We now can define the `CandidateExtractor` with the contexts to extract from, the matchers, and the throttler to use. 

In [13]:
from fonduer.candidates import CandidateExtractor


candidate_extractor = CandidateExtractor(session, [PartAttr], throttlers=[temp_throttler], parallelism=PARALLEL)

%time candidate_extractor.apply(docs, split=0)

[2019-04-01 21:58:37,688][INFO] fonduer.candidates.candidates:125 - Clearing table part_attr (split 0)
[2019-04-01 21:58:37,702][INFO] fonduer.utils.udf:54 - Running UDF...



CPU times: user 107 ms, sys: 47 ms, total: 154 ms
Wall time: 5.64 s


## Inspecting Candidates

Once you have run the `CandidateExtractor`, just like other elements of the Fonduer Data Model, you can query and inspect the `Candidates`.

In [14]:
train_cands = session.query(PartAttr).all()
print(f"Number of candidates: {len(train_cands)}")

Number of candidates: 2533


In [15]:
cand = train_cands[0]
print(cand)

PartAttr(Part(SpanMention("BC546", sentence=3063, chars=[0,4], words=[0,0])), Attr(SpanMention("200", sentence=4210, chars=[0,2], words=[0,0])))


Notice that our candidate is made up of two `Mentions`, one representing the `part`, and one the `attr` (maximum storage temperature in this case). We can look at each of those individually by simply calling their respective attributes

In [16]:
print(cand.part)
print(cand.attr)

Part(SpanMention("BC546", sentence=3063, chars=[0,4], words=[0,0]))
Attr(SpanMention("200", sentence=4210, chars=[0,2], words=[0,0]))


You can get the raw text of each candidate:

In [17]:
print(cand.part.context.get_span())
print(cand.attr.context.get_span())

BC546
200


Also notice that each `Span` contains the sentence it was found in, which provides you access to the full data model of the document in which it was found. You can explore the data model as was shown in the first tutorial. For example, we can access the Sentence:

In [18]:
print(cand.part.context.sentence)

Sentence (Doc: '112823', Sec: 0, Par: 0, Idx: 0, Text: 'BC546')


The document:

In [19]:
print(cand.part.context.sentence.document)

Document 112823


Or even the the document's `Tables`:

In [20]:
print(cand.part.context.sentence.document.tables)

[Table(Doc: 112823, Sec: 0, Position: 0), Table(Doc: 112823, Sec: 0, Position: 1), Table(Doc: 112823, Sec: 0, Position: 2), Table(Doc: 112823, Sec: 0, Position: 3), Table(Doc: 112823, Sec: 0, Position: 4), Table(Doc: 112823, Sec: 0, Position: 5), Table(Doc: 112823, Sec: 0, Position: 6), Table(Doc: 112823, Sec: 0, Position: 7), Table(Doc: 112823, Sec: 0, Position: 8)]


The full data model can be accessed from each candidate, which allows you to write powerful `Matchers` and `Throttlers` that leverage the structure and multimodality of your input documents.